In [22]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Set random seed for reproducibility
torch.manual_seed(42)

# Define graph
num_nodes = 100

# Fully connected undirected graph (without self-loops)
edge_index = torch.combinations(torch.arange(num_nodes), r=2).t()
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)  # Add reverse edges

# Node features: (100 x 16)
x = torch.randn(num_nodes, 16)

# Node labels: 3 classes (0, 1, 2)
y = torch.randint(0, 3, (num_nodes,))

# Randomly shuffle and split into 80 train, 20 test
perm = torch.randperm(num_nodes)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[perm[:80]] = True
test_mask = ~train_mask

# Create PyG Data object
data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)

# GCN model
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(16, 32)
        self.conv2 = GCNConv(32, 3)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Model, optimizer
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(50):
    model.train()
    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch % 10 == 0:
        print(f'Epoch {epoch:02d}, Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'Node Classification Accuracy: {acc:.4f}')


Epoch 00, Loss: 1.1077
Epoch 10, Loss: 1.0828
Epoch 20, Loss: 1.0824
Epoch 30, Loss: 1.0820
Epoch 40, Loss: 1.0818
Node Classification Accuracy: 0.2500


In [24]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Set seed for reproducibility
torch.manual_seed(42)

# 1. Define a graph
num_nodes = 100
edge_index = torch.combinations(torch.arange(num_nodes), r=2).t()
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)  # Make undirected

# 2. Node features
x = torch.randn(num_nodes, 16)

# 3. Create edge labels (e.g., binary classification: 0 or 1)
num_edges = edge_index.size(1)
edge_labels = torch.randint(0, 2, (num_edges,))

# 4. Split edges into train/test
perm = torch.randperm(num_edges)
train_edge_mask = torch.zeros(num_edges, dtype=torch.bool)
train_edge_mask[perm[:int(0.8 * num_edges)]] = True
test_edge_mask = ~train_edge_mask

# 5. Define the data object
data = Data(x=x, edge_index=edge_index)

# 6. GCN for node embeddings
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(16, 32)
        self.conv2 = GCNConv(32, 32)  # Final node embeddings

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# 7. Edge classifier (MLP on edge embeddings)
class EdgeClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn = GCN()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 2)  # 2 classes: 0 or 1
        )

    def forward(self, data, edge_index_to_classify):
        node_embeddings = self.gcn(data)  # Shape: [num_nodes, 32]
        src, dst = edge_index_to_classify  # Shape: [2, num_edges]
        edge_repr = torch.cat([node_embeddings[src], node_embeddings[dst]], dim=1)  # [num_edges, 64]
        return self.mlp(edge_repr)  # Logits for each edge

# 8. Initialize model and optimizer
model = EdgeClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 9. Training loop
for epoch in range(50):
    model.train()
    logits = model(data, edge_index[:, train_edge_mask])
    loss = F.cross_entropy(logits, edge_labels[train_edge_mask])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# 10. Evaluation
model.eval()
with torch.no_grad():
    test_logits = model(data, edge_index[:, test_edge_mask])
    preds = test_logits.argmax(dim=1)
    acc = (preds == edge_labels[test_edge_mask]).float().mean()
print(f"Edge Classification Accuracy: {acc:.4f}")


Epoch 0, Loss: 0.6938
Epoch 10, Loss: 0.6931
Epoch 20, Loss: 0.6931
Epoch 30, Loss: 0.6931
Epoch 40, Loss: 0.6931
Edge Classification Accuracy: 0.4914


In [25]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

# 1. Create a synthetic dataset of graphs
def create_graph_dataset(num_graphs=200, num_nodes=10):
    graphs = []
    for i in range(num_graphs):
        # Random features for each node
        x = torch.randn(num_nodes, 16)
        
        # Random edges: connect every node to a few others
        edge_index = torch.randint(0, num_nodes, (2, num_nodes * 2))
        
        # Binary graph label (0 or 1)
        y = torch.tensor([torch.randint(0, 2, (1,))], dtype=torch.long)
        
        # Each graph must have a 'batch' identifier later
        graph = Data(x=x, edge_index=edge_index, y=y)
        graphs.append(graph)
    return graphs

# Create dataset
dataset = create_graph_dataset()
train_dataset = dataset[:160]
test_dataset = dataset[160:]

# Load in batches
train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# 2. Define the model
class GraphClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(16, 32)
        self.conv2 = GCNConv(32, 32)
        self.classifier = torch.nn.Linear(32, 2)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # [num_graphs, 32]
        return self.classifier(x)       # [num_graphs, 2]

# 3. Training setup
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"{device} is available")
model = GraphClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 4. Training loop
for epoch in range(30):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = F.cross_entropy(out, batch.y.view(-1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# 5. Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.batch)
        pred = out.argmax(dim=1)
        correct += (pred == batch.y.view(-1)).sum().item()
        total += batch.num_graphs
print(f"Graph Classification Accuracy: {correct / total:.4f}")


/Users/yashvardhan/Documents/Desktop_Folders/ProjectsAndTutorials/testing_random_codes/venv/lib/python3.13/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


mps is available
Epoch 1, Loss: 3.5360
Epoch 2, Loss: 3.3183
Epoch 3, Loss: 3.1712
Epoch 4, Loss: 2.9595
Epoch 5, Loss: 2.7373
Epoch 6, Loss: 2.5246
Epoch 7, Loss: 2.3150
Epoch 8, Loss: 2.0865
Epoch 9, Loss: 1.8545
Epoch 10, Loss: 1.6020
Epoch 11, Loss: 1.3637
Epoch 12, Loss: 1.1319
Epoch 13, Loss: 0.9169
Epoch 14, Loss: 0.7259
Epoch 15, Loss: 0.5721
Epoch 16, Loss: 0.4581
Epoch 17, Loss: 0.5370
Epoch 18, Loss: 0.5486
Epoch 19, Loss: 0.8768
Epoch 20, Loss: 0.5797
Epoch 21, Loss: 0.3113
Epoch 22, Loss: 0.3200
Epoch 23, Loss: 0.2520
Epoch 24, Loss: 0.2858
Epoch 25, Loss: 0.2424
Epoch 26, Loss: 0.2187
Epoch 27, Loss: 0.1424
Epoch 28, Loss: 0.0916
Epoch 29, Loss: 0.1000
Epoch 30, Loss: 0.0755
Graph Classification Accuracy: 0.5500
